In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import sys
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    PeftConfig,
    
)

import fire
import torch
from datasets import load_dataset
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams

from transformers.generation.utils import GreedySearchDecoderOnlyOutput
from peft import PeftModel
from transformers import GenerationConfig

import json

import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')


%matplotlib inline
sns.set(rc={'figure.figsize':(10, 7)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)

#DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE='cuda:0'
torch.cuda.set_device(0)
DEVICE

'cuda:0'

## Load model

In [3]:
BASE_MODEL = "model/llama2-7B-hf"
model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map={'': 0}
    #device_map="auto",
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# BASE_MODEL = "model/llama2-7B-hf"
# LORA_WEIGHTS = "model/popular_hour_lr7e-5"

# model = LlamaForCausalLM.from_pretrained(
#     BASE_MODEL,
#     load_in_8bit=True,
#     torch_dtype=torch.float16,
#     device_map={'': 0},
#     local_files_only=True,
# )

# tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

# tokenizer.pad_token_id = (
#     0  # unk. we want this to be different from the eos token
# )
# tokenizer.padding_side = "left"

# model = PeftModel.from_pretrained(model, LORA_WEIGHTS, torch_dtype=torch.float16, device_map={'': 0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Generate Prompt

In [4]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""


def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

data = load_dataset("json", data_files="../data/alpaca-api-enrichment-popularhour-dataset.json")
CUTOFF_LEN=2048
train_val = data["train"].train_test_split(
    test_size=0.25, shuffle=True, seed=42
)
train_data = (
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].map(generate_and_tokenize_prompt)
)

Found cached dataset json (/home/bangzhao/.cache/huggingface/datasets/json/default-2f52a3215ac47957/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at /home/bangzhao/.cache/huggingface/datasets/json/default-2f52a3215ac47957/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-055101443ff61882.arrow and /home/bangzhao/.cache/huggingface/datasets/json/default-2f52a3215ac47957/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-14c98878887f1ab5.arrow
Loading cached processed dataset at /home/bangzhao/.cache/huggingface/datasets/json/default-2f52a3215ac47957/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-f2bb471707c5a906.arrow
Loading cached processed dataset at /home/bangzhao/.cache/huggingface/datasets/json/default-2f52a3215ac47957/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-695645c13f7fc956.arrow


## Training

In [5]:
LORA_R = 8 # attention head
LORA_ALPHA = 16 # alpha scaling
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

# BATCH_SIZE = 128
MICRO_BATCH_SIZE = 16
LEARNING_RATE = 7e-5
OUTPUT_DIR = "model/popular_hour_lr7e-5"

In [6]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [7]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    per_device_eval_batch_size=MICRO_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    num_train_epochs=10,
    logging_steps = int(len(train_val['train']) / MICRO_BATCH_SIZE),
    
    do_eval=True,
    fp16=True,
    optim="adamw_torch",
    output_dir=OUTPUT_DIR,
    load_best_model_at_end=True,
    
    #evaluation_strategy="steps",
    #save_strategy="steps",
    #max_steps= 80,
    #eval_steps=20,
    #save_steps=20
)

In [8]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [9]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model.config.use_cache = False

# old_state_dict = model.state_dict
# model.state_dict = (
#     lambda self, *_, **__: get_peft_model_state_dict(
#         self, old_state_dict()
#     )
# ).__get__(model, type(model))

model = torch.compile(model)

trainer.train(resume_from_checkpoint=True)
model.save_pretrained(OUTPUT_DIR)

Epoch,Training Loss,Validation Loss
9,0.639200,0.655149
10,0.636700,0.654468


In [10]:
from huggingface_hub import notebook_login
 
notebook_login()
 
model.push_to_hub("bangzhao/api_enrichment_llama2_lora_popular_hour", use_auth_token=True)

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bangzhao/api_enrichment_llama2_lora_popular_hour/commit/68b879fa760875dd2d81a998edc86f8f15fa5a81', commit_message='Upload model', commit_description='', oid='68b879fa760875dd2d81a998edc86f8f15fa5a81', pr_url=None, pr_revision=None, pr_num=None)

## Predicting

In [11]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
"""

def generate_response(prompt: str, model: PeftModel) -> GreedySearchDecoderOnlyOutput:
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to(DEVICE)

    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256,
        )
    
def format_response(response: GreedySearchDecoderOnlyOutput) -> str:
    decoded_output = tokenizer.decode(response.sequences[0])
    response = decoded_output.split("### Response:")[1].strip()
    return "\n".join(textwrap.wrap(response))

def ask_alpaca(prompt: str, model: PeftModel = model) -> str:
    prompt = generate_prompt(prompt)
    response = generate_response(prompt, model)
    return format_response(response)

In [12]:
ask_alpaca(val_data[1], model)

'Monday 08:00 to 16:00, Tuesday 08:00 to 16:00, Wednesday 08:00 to\n16:00, Thursday 08:00 to 16:00, Friday 08:00 to 16:00</s>'

### Load model and Lora weight

In [13]:
BASE_MODEL = "model/llama2-7B-hf"
LORA_WEIGHTS = "model/popular_hour_lr7e-5"
#ORA_WEIGHTS = "bangzhao/llama2_lora_api_enrichment_popularhour_predictor"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map={'': 0},
    #ocal_files_only=True,
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"


model = PeftModel.from_pretrained(model, LORA_WEIGHTS, torch_dtype=torch.float16, device_map={'': 0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
predictions = []

for idx, venue in enumerate(val_data):
    
    prediction = {}
    prediction['text'] = venue['input']
    prediction['truth'] = venue['output']
    prediction['generated'] = ask_alpaca(venue, model)
    
    predictions.append(prediction)
    
    if idx % 5 == 0:
        print(str(round(idx/len(val_data) * 100, 2)) + " %")
    if idx % 5 == 0:
        filename = f"../result/popularhour_predict_name_des_cate_pri_fea_tip.json"
        with open(filename, "w") as json_file:
            json.dump(predictions, json_file)

0.0 %
0.55 %
1.1 %
1.65 %
2.2 %
2.75 %
3.3 %
3.85 %
4.41 %
4.96 %
5.51 %
6.06 %
6.61 %
7.16 %
7.71 %
8.26 %
8.81 %
9.36 %
9.91 %
10.46 %
11.01 %
11.56 %
12.11 %
12.67 %
13.22 %
13.77 %
14.32 %
14.87 %
15.42 %
15.97 %
16.52 %
17.07 %
17.62 %
18.17 %
18.72 %
19.27 %
19.82 %
20.37 %
20.93 %
21.48 %
22.03 %
22.58 %
23.13 %
23.68 %
24.23 %
24.78 %
25.33 %
25.88 %
26.43 %
26.98 %
27.53 %
28.08 %
28.63 %
29.19 %


In [ ]:
filename = f"../result/popularhour_predict_name_des_cate_pri_fea_tip.json"
with open(filename, "w") as json_file:
    json.dump(predictions, json_file)